In [14]:
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain
import random

In [25]:
edges_df = pd.read_csv("../tmp/2011/el_no_missing.csv")
locations_df = pd.read_csv("../data/common/locations.csv")
nodes_df = pd.read_csv("../tmp/2011/nl_no_missing.csv")

In [26]:
nodes_df = pd.merge(left=nodes_df, right=locations_df, how='inner', left_on='country_iso3', right_on='Alpha-3code')
countries = nodes_df['country_iso3'].unique()

In [27]:
trade_data = np.zeros((len(countries), len(countries)))
for i in range(len(countries)):
    for j in range(len(countries)):
        temp = edges_df[
            (edges_df['source'] == countries[i]) & (edges_df['target'] == countries[j])
        ]
        if temp.shape[0] != 0:
            trade_data[i, j] += temp['weight']
            trade_data[j, i] += temp['weight']

In [28]:
g = nx.Graph()
for i in range(len(countries)):
    for j in range(i + 1, len(countries)):
        if trade_data[i, j] != 0:
            g.add_edge(countries[i], countries[j], weight = trade_data[i, j])

In [29]:
cols = list(nodes_df.columns)
cols.remove('country_iso3')

In [30]:
partition = community_louvain.best_partition(g)

In [31]:
nodal_attributes = dict()
for index, row in nodes_df.iterrows():
    nodal_attributes[row['country_iso3']] = dict()
    for col in cols:
        nodal_attributes[row['country_iso3']][col] = row[col]
    nodal_attributes[row['country_iso3']]['community'] = partition[row['country_iso3']]

In [32]:
nx.set_node_attributes(g, nodal_attributes)

In [33]:
countries_communities_df = pd.DataFrame(
    {
        'country': partition.keys(), 
        'community': partition.values()
    }
)

In [34]:
countries_communities_df.to_csv("../data/countries_communities.csv", index=False)